In [1]:
# 本代码由可视化策略环境自动生成 2022年4月15日 13:13
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：初始化函数，只执行一次
def m8_initialize_bigquant_run(context):

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    context.stock_count = 1
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    #context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, stock_count)])
    # 每只股票的权重平均分配
    context.stock_weights = 1/context.stock_count
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 1
    context.options['hold_days'] = 0


# 回测引擎：每日数据处理函数，每天执行一次
def m8_handle_data_bigquant_run(context, data):
    today = data.current_dt.strftime('%Y-%m-%d')
    equities = {e.symbol: p for e, p in context.portfolio.positions.items() if p.amount>0}
    stock_now = len(equities); #获取当前持仓股票数量
    stock_count = context.stock_count
    
    # 按日期过滤得到今日的预测数据
    # 加载预测数据
    df = context.options['data'].read_df()
    df_today = df[df.date == data.current_dt.strftime('%Y-%m-%d')]
    df_today.set_index('instrument')
    
    

    #------------------------------------------止损模块START--------------------------------------------
    
    # 新建当日止损股票列表是为了handle_data 策略逻辑部分不再对该股票进行判断
    stoploss_stock = []
    stopwin_stock = []
    now_stock = []
    if len(equities) > 0:
        for i in equities.keys():
            now_stock.append(i)
            try :
                amount_zfs=list(df_today[df_today.instrument==i]['amount_zf'])
                amount_zf=amount_zfs[0]
                amount_bls=list(df_today[df_today.instrument==i]['amount_bl'])
                amount_bl=amount_bls[0]
                mc1s=list(df_today[df_today.instrument==i]['mc1'])
                mc1=mc1s[0]
                mc3s=list(df_today[df_today.instrument==i]['mc3'])
                mc3=mc3s[0]
                mc4s=list(df_today[df_today.instrument==i]['mc4'])
                mc4=mc4s[0]
                gzxzs=list(df_today[df_today.instrument==i]['gzxz'])
                gzxz=gzxzs[0]
                hasMdins=list(df_today[df_today.instrument==i]['hasMdin'])
                hasMdin=hasMdins[0]
                d2s=list(df_today[df_today.instrument==i]['d2'])
                d2=d2s[0]
            except :
                amount_bl=0
                amount_zf=0
                mc1=0
                gzxz=0
                mc3=0
                mc4=0
                hasMdin=0
                d2=0
            stock_market_price = data.current(context.symbol(i), 'price')  # 最新市场价格
            stock_market_today_high = data.current(context.symbol(i), 'high') #今日最高价      
            stock_market_today_close = data.current(context.symbol(i), 'close') #今日收盘价
            last_sale_date = equities[i].last_sale_date   # 上次交易日期
            last_cost_price = equities[i].cost_basis # 上次交易金额
            delta_days = data.current_dt - last_sale_date  
            hold_days = delta_days.days # 持仓天数
            # 建仓以来的最高价
            highest_price_since_buy = data.history(context.symbol(i), 'high', hold_days, '1d').max()
            # 建仓以来的收盘价的最高价
            highclose_price_since_buy = data.history(context.symbol(i), 'close', hold_days, '1d').max()
            # 确定止损位置
            stoploss_line = highest_price_since_buy - highest_price_since_buy * 0.05
            
            # 确定止盈位置
            stopwin_line = last_cost_price * 1.1
            # 最高收益
            high_return = (highclose_price_since_buy-last_cost_price)/last_cost_price
            # 今日回落（最高价-收盘价）/收盘价
            hui_luo = (stock_market_today_high - stock_market_price)/stock_market_price
            if(i=='603909.SHA'):
                print('日期:',today,'high_return:',high_return)   
            #record('止损位置', stoploss_line)
            # 如果价格下穿止损位置
            if stock_market_price < stoploss_line:
                if(i=='002735.SZA'):
                    print('日期:',today,'最高价:',highest_price_since_buy)
                    print('日期:',today,'止损价:',stoploss_line)
                    print('日期:',today,'当前价:',stock_market_price)
                    print('日期:',today,'建仓价:',last_cost_price)
                #context.order_target_percent(context.symbol(i), 0)
                context.order_target(context.symbol(i), 0)
                stoploss_stock.append(i)
                stock_now = stock_now -1
            #收益达6个点后，冲高回落时卖出
            elif high_return>0.06 and hui_luo>0.04 and amount_zf<1.5 and amount_bl<2:
                context.order_target(context.symbol(i), 0)
                stoploss_stock.append(i)
                stock_now = stock_now -1
                print('日期：', today, '股票：', context.symbol(i), 'huiluo:',hui_luo,'high_return:',high_return)
                print('日期：', today, '股票：', i, '出现冲高回落止损状况')
            #当天涨幅或相对前一天涨幅大于5个点，且成交量下降时卖出
            elif mc1==1 :
                context.order_target(context.symbol(i), 0)
                stoploss_stock.append(i)
                stock_now = stock_now -1
                print('日期：', today, '股票：', i, '大涨且成交量下降时卖出')
            #大涨后，出现高幅振汤且小涨卖出
            elif high_return>0.06 and gzxz==1 :
                context.order_target(context.symbol(i), 0)
                stoploss_stock.append(i)
                stock_now = stock_now -1
                print('日期：', today, '股票：', i, '大涨后出现高振小涨卖出')
            #大涨后，出现下跌时卖出
            elif mc3==1 and high_return>0.08:
                context.order_target(context.symbol(i), 0)
                stoploss_stock.append(i)
                stock_now = stock_now -1
                print('日期：', today, '股票：', i, '大涨后小跌时卖出')
            #高振幅小涨时卖出
            elif mc4==1:
                context.order_target(context.symbol(i), 0)
                stoploss_stock.append(i)
                stock_now = stock_now -1
                print('日期：', today, '股票：', i, '高振幅小涨时卖出')
            elif hasMdin==1:   
                #获取M顶的最大值
                if(i=='002633.SZA'):
                    mDinValue = data.history(context.symbol(i), 'close',29-int(d2), '1d')[0]
                    #M顶出现后的最高值
                    mDinLaterMax = data.history(context.symbol(i), 'close',29-int(d2), '1d').max()
                    print('日期：', today, '股票：', i, 'M顶值为：',mDinValue,'M顶后的最大值：',mDinLaterMax)
                    
            #elif stock_market_price > stopwin_line:
            #    context.order_target(context.symbol(i), 0)
            #    stopwin_stock.append(i)
            #    stock_now = stock_now -1
    if len(stoploss_stock)>0:
        print('日期：', today, '股票：', stoploss_stock, '出现跟踪止损状况')
    #if len(stopwin_stock)>0:
    #    print('日期：', today, '股票：', stopwin_stock, '出现跟踪止盈状况')
    
    #-------------------------------------------止损模块END---------------------------------------------   
    
    try:
        buy_list = context.daily_buy_stock[today]
    except:
        buy_list = []

    
    # 1. 资金分配
    #is_staging = context.trading_day_index < context.options['hold_days'] # 是否在建仓期间（前 hold_days 天） 
    stock_cash = context.portfolio.portfolio_value/stock_count
    #cash_avg = context.portfolio.portfolio_value
    #cash_for_buy = min(context.portfolio.cash,  stock_cash)
    #cash_for_sell = cash_avg - (context.portfolio.cash - cash_for_buy)
    
    positions = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.perf_tracker.position_tracker.positions.items()}
    
    # 2. 生成卖出订单：hold_days天之后才开始卖出；
    try:
        sell_stock_list = context.daily_sell_stock[today]  # 当日符合卖出条件的股票
    except:
        sell_stock_list = []
    # 需要卖出的股票:已有持仓中符合卖出条件的股票
    stock_to_sell = [i for i in positions if i in sell_stock_list]
    #print('日期：', today, '股票：', stock_to_sell, '需要卖出的股票')
    sell_stock = []
    
     # 卖出
    for instrument in stock_to_sell:
        # 如果该股票停牌，则没法成交。因此需要用can_trade方法检查下该股票的状态
        # 如果返回真值，则可以正常下单，否则会出错
        # 因为stock是字符串格式，我们用symbol方法将其转化成平台可以接受的形式：Equity格式
        if data.can_trade(context.symbol(instrument)):
            # order_target_percent是平台的一个下单接口，表明下单使得该股票的权重为0，即卖出全部股票，可参考回测文档
            context.order_target(context.symbol(instrument), 0)
            sell_stock.append(instrument)
            stock_now = stock_now -1
            if(instrument=='002735.SZA'):
                    print('日期:',today,'卖出1:',instrument)
            
    #if not is_staging :
    if 1==1 :    
        if len(equities) > 0:
            for i in equities.keys():
                last_sale_date = equities[i].last_sale_date	# 上次交易日期
                delta_days = data.current_dt - last_sale_date  
                hold_days = delta_days.days # 持仓天数
                print('日期:',today,'hold_days:',hold_days,'股票:',i)
                if hold_days >= context.options['hold_days'] and i not in buy_list + stoploss_stock + stopwin_stock + sell_stock:
                    print('日期:',today,'卖出2:',i)
                    context.order_target(context.symbol(i), 0)
                    sell_stock.append(i)
                    stock_now = stock_now -1
                    if(i=='002735.SZA'):
                        print('日期:',today,'卖出2:',i)
                    #print('日期：', today, '股票：', i, ' 卖出')
                 
# 3. 生成买入订单
    buy_num = stock_count - stock_now
    #if is_staging :
    #    buy_num = 1
    if len(buy_list)>0:
        print('日期：', today, '选出股票数量：', len(buy_list))
    if buy_num>0 and len(buy_list)>0 :
        # 不再买入已经轮仓卖出和移动止损的股票,以防止出现空头持仓
        buy_instruments = [i for i in buy_list if i not in sell_stock + stoploss_stock + stopwin_stock + now_stock][:buy_num]
        for i, instrument in enumerate(buy_instruments):
            current_price = data.current(context.symbol(instrument), 'price')
            #过滤不买入的
            try :
                hasMdins=list(df_today[df_today.instrument==instrument]['hasMdin'])
                hasMdin=hasMdins[0]
                d2s=list(df_today[df_today.instrument==instrument]['d2'])
                d2=d2s[0]
                zs_a1=list(df_today[df_today.instrument==instrument]['zs_a1'])[0]
                zs_a2=list(df_today[df_today.instrument==instrument]['zs_a2'])[0]
                zs_bmy1=list(df_today[df_today.instrument==instrument]['zs_bmy1'])[0]            
            except :
                hasMdin=0
                d2=0
                zs_bmy1=0
              #  bmy4=0
            if hasMdin==1: #需要优化M顶的逻辑
                mDinValue = data.history(context.symbol(instrument), 'close',29-int(d2), '1d')[0]
                #M顶出现后的最高值
                mDinLaterMax = data.history(context.symbol(instrument), 'close',29-int(d2), '1d').max()
                mDinLaterMin = data.history(context.symbol(instrument), 'close',29-int(d2), '1d').min()
                #当出现了M顶，且M顶未被突破，当前的价格低于M顶，且当前价格接近M顶的位置，不买入
                if mDinLaterMax/mDinValue<1.03 and current_price<mDinValue and mDinValue/current_price<1.04 and mDinValue/mDinLaterMin>1.07:
                    print('日期:',today,'买入时过滤有顶部压力的股票:',instrument)
                    continue
            if zs_bmy1==3:
                    print('日期:',today,'买入时过滤指数不买入股票:',instrument,'指数a1:',zs_a1,'指数a2:',zs_a2)
                    #continue
            cash_for_buy = min(context.portfolio.cash,  stock_cash)
            if cash_for_buy>0 and data.can_trade(context.symbol(instrument)):           
                amount = math.floor(cash_for_buy / current_price / 100) * 100
                context.order(context.symbol(instrument), amount)
                #if(instrument=='002735.SZA'):
                print('日期:',today,'买入:',instrument)
            else :
                print('日期:',today,'无资金或不能交易未买入:',instrument)
# 回测引擎：准备数据，只执行一次
def m8_prepare_bigquant_run(context):
    # 加载预测数据
    df = context.options['data'].read_df()
    # 函数：求满足开仓条件的股票列表
    def open_pos_con(df):
        return list(df[df['buy_condition']>0].instrument)
    # 函数：求满足平仓条件的股票列表
    def close_pos_con(df):
        return list(df[df['sell_condition']>0].instrument)
    
    # 每日卖出股票的数据框
    context.daily_sell_stock= df.groupby('date').apply(close_pos_con)  
    # 每日买入股票的数据框
    context.daily_buy_stock= df.groupby('date').apply(open_pos_con)  


    

# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m8_before_trading_start_bigquant_run(context, data):
    pass


m1 = M.instruments.v2(
    start_date='2019-01-01',
    end_date=T.live_run_param('trading_date', '2022-05-01'),
    market='CN_STOCK_A',
    instrument_list='        ',
    max_count=0
)

m2 = M.input_features.v1(
    features="""isXiaDie0=where((return_0<1),1,0)
#当天A股下跌数
xd_num=group_sum(date, isXiaDie0)
isZhangtToday=where((return_0>1.09)&(close_0==high_0),1,0)
isDietToday=where((return_0<0.91)&(close_0==low_0),1,0)
return0=return_0
priceLowBl10=close_0/ts_min(close_0,10)
zgzzf=high_0/close_1
zdzzf=low_0/close_1
kp1=(open_0-close_1)/close_1
kp=shift((open_0-close_1)/close_1,-1)
zd=shift((low_0-close_1)/close_1,-1)
zg=shift((high_0-close_1)/close_1,-1)

#逻辑一：大盘大跌情况下的涨停票后续效果
#myls=where((xd_num>3500)&(isZhangtToday==1),1,0)
#逻辑二：大盘大跌情况下的跌停票后续效果
#myls=where((xd_num>3500)&(isDietToday==1),1,0)
#逻辑三：大盘大跌情况下的跌停票后续效果，精选个股
myls=where((xd_num>3500)&(isDietToday==1)&(priceLowBl10<1.05)&(kp1>-0.05),1,0)

shouyi=(shift(open_0,-2)-shift(open_0,-1))/shift(open_0,-1)
shouyi2=(shift(close_0,-2)-shift(open_0,-1))/shift(open_0,-1)

kp=where((kp<-0.096),1,kp)
#zhangf=(close_0-open_0)/open_0

#附合这个选股逻辑，第二天开盘执行买入
buy_condition=where(myls==1,1,0)
#卖出不做特别处理，按三天收盘执行卖出
sell_condition=where(myls==2,1,0)"""
)

m5 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m2.data,
    start_date='',
    end_date='',
    before_start_days=0,
    m_cached=False
)

m10 = M.instruments.v2(
    start_date='2021-01-02',
    end_date='2022-05-01',
    market='CN_STOCK_A',
    instrument_list='000002.HIX',
    max_count=0
)

m12 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
zs_zhangf=(close-open)/open
zs_open=open/shift(close,1)
zs_huiluo=(high-close)/close
zs_huishen=(close-low)/low
zs_zhenf=(high-low)/shift(close,1)
zs_volume_zf=volume/shift(volume,1)
zs_return_0=close/shift(close,1)
zs_return_1=shift(zs_return_0,1)
zs_return_2=shift(zs_return_0,2)
zs_max10=ts_max(close,10)
zs_max10d=ts_argmax(close,10)
zs_max30=ts_max(close,30)
#zs_max30d=ts_argmax(close,30)
zs_min10=ts_min(close,10)
#zs_min10d=ts_argmin(close,10)
zs_min30=ts_min(close,30)
#zs_min30d=ts_argmin(close,30)
zs_priceHighBl10=close/zs_max10
zs_priceLowBl10=close/zs_min10
zs_priceHighBl30=close/zs_max30
zs_priceLowBl30=close/zs_min30"""
)

m4 = M.use_datasource.v1(
    instruments=m10.data,
    features=m12.data,
    datasource_id='bar1d_index_CN_STOCK_A',
    start_date='',
    end_date=''
)

m13 = M.derived_feature_extractor.v3(
    input_data=m4.data,
    features=m12.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=True,
    user_functions={}
)

m18 = M.instruments.v2(
    start_date='2021-01-02',
    end_date='2022-05-01',
    market='CN_STOCK_A',
    instrument_list='000002.HIX',
    max_count=0
)

m19 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
close
"""
)

m17 = M.use_datasource.v1(
    instruments=m18.data,
    features=m19.data,
    datasource_id='bar1d_index_CN_STOCK_A',
    start_date='',
    end_date=''
)

m20 = M.data_join.v3(
    input_1=m13.data,
    input_2=m17.data,
    on='date',
    how='left',
    sort=False
)

m14 = M.data_join.v3(
    input_1=m5.data,
    input_2=m20.data,
    on='date',
    how='left',
    sort=False
)

m7 = M.derived_feature_extractor.v3(
    input_data=m14.data,
    features=m2.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=True,
    user_functions={}
)

m6 = M.chinaa_stock_filter.v1(
    input_data=m7.data,
    index_constituent_cond=['全部'],
    board_cond=['上证主板', '深证主板'],
    industry_cond=['全部'],
    st_cond=['正常'],
    delist_cond=['非退市'],
    output_left_data=False
)

m11 = M.dropnan.v2(
    input_data=m6.data
)

m9 = M.sort.v4(
    input_ds=m11.data,
    sort_by='kp',
    group_by='date',
    keep_columns='--',
    ascending=True
)

m3 = M.filter.v3(
    input_data=m9.sorted_data,
    expr='myls==1',
    output_left_data=False
)

m8 = M.trade.v4(
    instruments=m1.data,
    options_data=m3.data,
    start_date='',
    end_date='',
    initialize=m8_initialize_bigquant_run,
    handle_data=m8_handle_data_bigquant_run,
    prepare=m8_prepare_bigquant_run,
    before_trading_start=m8_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=200000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='真实价格',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)


[2022-03-01 13:12:40.389223] INFO: moduleinvoker: instruments.v2 开始运行..

[2022-03-01 13:12:40.409617] INFO: moduleinvoker: 命中缓存

[2022-03-01 13:12:40.411728] INFO: moduleinvoker: instruments.v2 运行完成[0.022526s].

[2022-03-01 13:12:40.421139] INFO: moduleinvoker: input_features.v1 开始运行..

[2022-03-01 13:12:40.431099] INFO: moduleinvoker: 命中缓存

...

[2022-03-01 13:14:19.692239] INFO: Performance: Simulated 518 trading days out of 518.

[2022-03-01 13:14:19.694969] INFO: Performance: first open: 2020-01-02 09:30:00+00:00

[2022-03-01 13:14:19.696390] INFO: Performance: last close: 2022-02-23 15:00:00+00:00

[2022-03-01 13:14:29.551897] INFO: moduleinvoker: backtest.v8 运行完成[39.054719s].

[2022-03-01 13:14:29.554157] INFO: moduleinvoker: trade.v4 运行完成[41.150615s].